In [1]:
# %matplotlib widget
%matplotlib inline

10

In [2]:
# 1) Wipe out all Python variables
%reset -f
# 2) Force Python’s garbage collector to run
import gc
gc.collect()

import importlib
from libs import trades, plots, params, models
importlib.reload(trades)
importlib.reload(plots)
importlib.reload(params)
importlib.reload(models)

import pandas as pd
import numpy  as np
import math

import datetime as dt
import os
from typing import Sequence, List, Tuple, Optional, Union

import torch
import torch.nn as nn
import torch.nn.functional as Funct
from torch_lr_finder import LRFinder

from tqdm import tqdm

In [3]:

ticker         = params.ticker
look_back      = params.look_back 
features_cols  = params.features_cols
label_col      = params.label_col
save_path      = params.save_path

date = dt.datetime.now().strftime("%Y-%m-%d %H:%M")
regular_start_pred  = params.regular_start_pred

# USE GPU if available, otherwise fallback to CPU
device = params.device
print("Using device:", device)

fin_csv_path  = save_path / f"{ticker}_final.csv"
df = pd.read_csv(fin_csv_path, index_col=0, parse_dates=True)
df

Using device: cuda


,open,high,low,close,volume,r_1,r_5,r_15,vol_15,volume_spike,vwap_dev,rsi_14,bid,ask,signal_smooth
2014-04-03 12:06:00,28.644845,28.644845,28.644845,28.644845,4580.0,-0.000180,-0.000180,-0.000180,0.000046,0.568641,-0.000177,0.000000,28.636251,28.653438,0.162925
2014-04-03 12:07:00,28.639690,28.639690,28.639690,28.639690,4540.0,-0.000180,-0.000360,-0.000360,0.000063,0.570338,-0.000352,0.000000,28.631098,28.648282,0.164530
2014-04-03 12:08:00,28.634534,28.634534,28.634534,28.634534,4500.0,-0.000180,-0.000540,-0.000540,0.000075,0.574408,-0.000524,0.000000,28.625944,28.643125,0.166141
2014-04-03 12:09:00,28.629379,28.629379,28.629379,28.629379,4460.0,-0.000180,-0.000720,-0.000720,0.000082,0.581017,-0.000694,0.000000,28.620791,28.637968,0.167757
2014-04-03 12:10:00,28.624224,28.624224,28.624224,28.624224,4420.0,-0.000180,-0.000900,-0.000900,0.000088,0.590413,-0.000862,0.000000,28.615637,28.632811,0.169381
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2025-06-17 20:56:00,176.090000,176.160000,176.010000,176.050000,126589.0,-0.000284,-0.000681,-0.001297,0.000443,1.610125,1.217786,39.325843,175.997200,176.102800,0.000000
2025-06-17 20:57:00,176.050000,176.080000,175.890000,175.930000,125176.0,-0.000682,-0.000398,-0.001761,0.000469,1.478464,1.216271,34.328358,175.877200,175.982800,0.000000
2025-06-17 20:58:00,175.930000,175.975000,175.860000,175.890000,200235.0,-0.000227,-0.001023,-0.002016,0.000468,2.096227,1.215761,29.648241,175.837200,175.942800,0.000000
2025-06-17 20:59:00,175.900000,175.970000,175.800000,175.930000,393414.0,0.000227,-0.001136,-0.002073,0.000464,3.295458,1.216254,25.925926,175.877200,175.982800,0.000000


In [4]:
X, y, raw_close, raw_bid, raw_ask = models.build_lstm_tensors(
    df=df,
    look_back=look_back,
    features_cols=features_cols,
    label_col=label_col,
    regular_start=regular_start_pred
)

# 1)
print(X.shape) # we use 'm' features and 'n' previous look back values to predict each 1 label
print(y.shape) 


# 2) quick shapes
print("Shapes:")
print("  X         =", X.shape,    "(samples, features, look_back)")
print("  y         =", y.shape,    "(samples,)")
print("  raw_close =", raw_close.shape)
print("  raw_bid   =", raw_bid.shape)
print("  raw_ask   =", raw_ask.shape)

# # 3) rebuild the list of label‐timestamps (window‐ends)
# ends = []
# for day, day_df in df.groupby(df.index.normalize(), sort=False):
#     day_df = day_df.sort_index()
#     # candidate ends at positions look_back .. end
#     idxs = day_df.index[look_back:]
#     # only keep those at/after regular_start_pred
#     mask = [t >= regular_start_pred for t in idxs.time]
#     ends.extend(idxs[mask])

# # 4) show first few ends
# print("\nFirst 5 window‐end timestamps:")
# for ts in ends[:5]:
#     print(" ", ts)

# # 5) show exactly which minutes X[0] contains
# first_end   = ends[0]
# first_start = first_end - pd.Timedelta(minutes=look_back)
# print(f"\nFirst window covers {look_back} bars from")
# print(f"  {first_start}  →  {first_end - pd.Timedelta(minutes=1)}")
# print(f"and predicts the bar at {first_end}")

# print("\nThose bars (timestamps):")
# print(pd.date_range(first_start, first_end - pd.Timedelta(minutes=1), freq="1min"))
# # -------------------------------------------------------------------

# y_np         = y.cpu().numpy()
# print("First 5 values y:",         y_np[:5])
# print("First 5 signal_smooth values, on regular trade time:")
# df.signal_smooth.iloc[look_back:look_back+5]

torch.Size([1174595, 120, 12])
torch.Size([1174595])
Shapes:
  X         = torch.Size([1174595, 120, 12]) (samples, features, look_back)
  y         = torch.Size([1174595]) (samples,)
  raw_close = torch.Size([1174595])
  raw_bid   = torch.Size([1174595])
  raw_ask   = torch.Size([1174595])

First 5 window‐end timestamps:
  2014-04-03 14:06:00
  2014-04-03 14:07:00
  2014-04-03 14:08:00
  2014-04-03 14:09:00
  2014-04-03 14:10:00

First window covers 120 bars from
  2014-04-03 12:06:00  →  2014-04-03 14:05:00
and predicts the bar at 2014-04-03 14:06:00

Those bars (timestamps):
DatetimeIndex(['2014-04-03 12:06:00', '2014-04-03 12:07:00',
               '2014-04-03 12:08:00', '2014-04-03 12:09:00',
               '2014-04-03 12:10:00', '2014-04-03 12:11:00',
               '2014-04-03 12:12:00', '2014-04-03 12:13:00',
               '2014-04-03 12:14:00', '2014-04-03 12:15:00',
               ...
               '2014-04-03 13:56:00', '2014-04-03 13:57:00',
               '2014-04-03 13:

2014-04-03 14:06:00    0.287953
2014-04-03 14:07:00    0.289543
2014-04-03 14:08:00    0.291579
2014-04-03 14:09:00    0.294060
2014-04-03 14:10:00    0.295071
Name: signal_smooth, dtype: float64

In [5]:
# Split into train/val/test by calendar day
(X_tr, y_tr), \
(X_val, y_val), \
(X_te, y_te, raw_close_te, raw_bid_te, raw_ask_te), \
samples_per_day, day_id_tr, day_id_val, day_id_te = models.chronological_split(
    X, y, raw_close, raw_bid, raw_ask, df,
    look_back   = look_back,
    regular_start   = regular_start_pred,
    train_prop  = params.train_prop,
    val_prop    = params.val_prop,
    train_batch = params.hparams['TRAIN_BATCH']
)

# 1) Print shapes of all tensors
print("Shapes:")
print("  X_tr        =", X_tr.shape)
print("  y_tr        =", y_tr.shape)
print("  raw_close_te=", raw_close_te.shape)
print("  raw_bid_te  =", raw_bid_te.shape)
print("  raw_ask_te  =", raw_ask_te.shape)

# # 2) Print number of days in each split
# n_tr_days = torch.unique(day_id_tr).numel()
# n_val_days= torch.unique(day_id_val).numel()
# n_te_days = torch.unique(day_id_te).numel()
# print(f"\nDays: train={n_tr_days}, val={n_val_days}, test={n_te_days}")

# # 3) Print number of windows in each split
# print(f"Windows: train={X_tr.shape[0]}, val={X_val.shape[0]}, test={X_te.shape[0]}")

# # 4) List the first few window‐end timestamps
# ends = []
# for day, day_df in df.groupby(df.index.normalize(), sort=False):
#     ts = day_df.index[look_back:]
#     ends.extend(ts[ts.time >= regular_start_pred])
# first_ends = ends[:5]
# print("\nFirst 5 window‐end times:", first_ends)

# # 5) Show exactly which minutes X_tr[0] covers, and where y_tr[0] sits
# first_end   = first_ends[0]
# first_start = first_end - pd.Timedelta(minutes=look_back)
# # input bars = [first_start … first_end − 1min]
# print(f"\nX_tr[0] covers bars from {first_start} to {first_end - pd.Timedelta(minutes=1)}")
# print("Those timestamps:")
# print(pd.date_range(first_start, first_end - pd.Timedelta(minutes=1), freq="1min"))
# print(f"y_tr[0] (and raw_close_te[0]) is the bar at {first_end}")


KeyboardInterrupt: 

In [ ]:

# -----------------------------------------------------------------------------
#  Build DataLoaders over calendar‐days
# -----------------------------------------------------------------------------
train_loader, val_loader, test_loader = models.split_to_day_datasets(
    # Training split arrays (from chronological_split)
    X_tr, y_tr, day_id_tr,
    # Validation split arrays
    X_val, y_val, day_id_val,
    # Test split arrays + raw prices for post‐tracking
    X_te, y_te, day_id_te, raw_close_te, raw_bid_te, raw_ask_te,
    # Original minute‐bar DataFrame for weekday mapping
    df=df,
    train_batch=params.hparams['TRAIN_BATCH'],
    train_workers=params.hparams['NUM_WORKERS']
)


In [ ]:

# -----------------------------------------------------------------------------
# Instantiate the stateful DualMemoryLSTM & move to device
# -----------------------------------------------------------------------------
model = models.DualMemoryLSTM(
    n_feats        = len(features_cols),                          
    short_units    = params.hparams['SHORT_UNITS'],    
    long_units     = params.hparams['LONG_UNITS'],     
    dropout_short  = params.hparams['DROPOUT_SHORT'],  
    dropout_long   = params.hparams['DROPOUT_LONG'],   
    att_heads      = params.hparams['ATT_HEADS'],
    att_drop       = params.hparams['ATT_DROPOUT']
)
model.to(device)   # place model parameters on GPU or CPU as specified

model

In [ ]:

# -----------------------------------------------------------------------------
# Compute plateau_sched timing parameters
# -----------------------------------------------------------------------------
# Total training samples = total windows in X_tr (one window per row)
n_train_samples = X_tr.shape[0]

# How many optimizer steps (day‐bundles) constitute one epoch?
steps_per_epoch = len(train_loader)


# -----------------------------------------------------------------------------
# Build optimizer, LR scheduler, AMP scaler, and gradient‐clip norm
# -----------------------------------------------------------------------------
optimizer, plateau_sched, cosine_sched, scaler, clipnorm = models.make_optimizer_and_scheduler(
    model,
    initial_lr        = params.hparams['INITIAL_LR'],       
    weight_decay      = params.hparams['WEIGHT_DECAY'],     
    clipnorm          = params.hparams['CLIPNORM']   
)

optimizer

In [ ]:
############################################
# function to find the optimal learning rate
############################################

# # 1) Move model to CPU and build a fresh optimizer (no scheduler metadata)
# model_cpu = model.cpu()
# optimizer_cpu = torch.optim.AdamW(
#     model_cpu.parameters(),
#     lr=1e-3,        # placeholder; the finder will override this
#     weight_decay=5e-4
# )

# # 2) Create a tiny DataLoader (batch_size=1) to save memory
# small_loader = DataLoader(
#     train_loader.dataset,
#     batch_size=1,
#     shuffle=True,
#     num_workers=0
# )

# # 3) Define an aligned MSE that permutes/expands your [1,1,D] or [D,1,1]
# #    target → [D, T, 1] to match output shape exactly.
# def aligned_mse(output, target):
#     # output: [D, T, 1]
#     # target might come in as [D,1,1] or [1,1,D]
#     tgt = target

#     # Case A: target == [D, 1, 1] → expand middle dim to T
#     if tgt.dim() == 3 and tgt.shape[0] == output.shape[0] \
#        and tgt.shape[1] == 1 and tgt.shape[2] == 1:
#         tgt = tgt.expand(-1, output.size(1), -1)

#     # Case B: target == [1, 1, D] → permute to [D,1,1] then expand
#     elif tgt.dim() == 3 and tgt.shape[0] == 1 \
#          and tgt.shape[1] == 1 and tgt.shape[2] == output.shape[0]:
#         # permute (0,1,2) → (2,1,0) to get [D,1,1]
#         tgt = tgt.permute(2, 1, 0)
#         tgt = tgt.expand(-1, output.size(1), -1)

#     else:
#         # fallback: broadcast to exactly output.shape
#         tgt = tgt.expand(output.shape)

#     return Funct.mse_loss(output, tgt)

# # 4) Free any lingering GPU memory
# gc.collect()
# torch.cuda.empty_cache()

# # 5) Run the LR‐Finder on CPU for just 30 mini‐batches
# lr_finder = LRFinder(
#     model_cpu,
#     optimizer_cpu,
#     aligned_mse,
#     device="cpu"
# )
# lr_finder.range_test(
#     small_loader,
#     end_lr=1,     # maximum LR to try
#     num_iter=30   # number of batches
# )
# lr_finder.plot()   # examine loss vs. LR curve
# lr_finder.reset()  # restore original model & optimizer states

# # 6) Move model back to GPU for your main training
# model = model_cpu.to("cuda")


In [ ]:

# -----------------------------------------------------------------------------
# Count how many calendar days we see each epoch and Compute baseline RMSE on validation (zero forecast)
# -----------------------------------------------------------------------------
n_train_days = len(train_loader.dataset)  # dataset length = # unique days
print(f"Training sees {n_train_days} calendar days per epoch\n")

baseline_val_rmse = models.naive_rmse(val_loader)
print(f"Baseline (zero‐forecast) RMSE on validation = {baseline_val_rmse:.6f}")


# -----------------------------------------------------------------------------
# Run the custom stateful training loop
# -----------------------------------------------------------------------------
best_val_rmse  = models.custom_stateful_training_loop(
    model               = model,
    optimizer           = optimizer,
    cosine_sched        = cosine_sched,
    plateau_sched       = plateau_sched,
    scaler              = scaler,
    train_loader        = train_loader,
    val_loader          = val_loader,
    max_epochs          = params.hparams['MAX_EPOCHS'],
    early_stop_patience = params.hparams['EARLY_STOP_PATIENCE'],
    baseline_val_rmse   = baseline_val_rmse,
    clipnorm            = clipnorm,
    device              = device
)

# -----------------------------------------------------------------------------
# Final reporting: best RMSE and relative improvement
# -----------------------------------------------------------------------------
print(f"\nChampion validation RMSE = {best_val_rmse:.6f}")

improvement = 100.0 * (1.0 - best_val_rmse / baseline_val_rmse)
print(f"Improvement over zero‐baseline = {improvement:5.1f}%")
